# Using AI21 Paraphrase on SageMaker through Model Packages

This sample notebook shows you how to deploy **AI21 Paraphrase** using Amazon SageMaker.


--------------------
## <font color='orange'>Important:</font>
Please visit model detail page in <a href="https://aws.amazon.com/marketplace/pp/prodview-6ivt5p34sljua">https://aws.amazon.com/marketplace/pp/prodview-6ivt5p34sljua</a> to learn more. <font color='orange'>If you do not have access to the link, please contact account admin for the help.</font>

You will find details about the model including pricing, supported region, and end user license agreement. To use the model, please click “<font color='orange'>Continue to Subscribe</font>” from the detail page, come back here and learn how to deploy and inference.


-------------------


## Pre-requisites:
1. Before running this notebook, please make sure you got this notebook from the model catalog on SageMaker AWS Management Console.
1. **Note**: This notebook contains elements which render correctly in Jupyter interface. Open this notebook from an Amazon SageMaker Notebook Instance or Amazon SageMaker Studio.
1. Ensure that IAM role used has **AmazonSageMakerFullAccess**.
1. This notebook is intended to work with **boto3 v1.25.4** or higher.

## Contents:
1. [Select model package](#1.-Select-model-package)
1. [Create an endpoint and perform real-time inference](#2.-Create-an-endpoint-and-perform-real-time-inference)
   1. [Create an endpoint](#A.-Create-an-endpoint)
   1. [Paraphrase a single sentence](#B.-Paraphrase-a-single-sentence)
   1. [Paraphrase with a ceratin style](#C.-Paraphrase-with-a-ceratin-style)
   1. [Add context to enhance the paraphrasing](#D.-Add-context-to-enhance-the-paraphrasing)
1. [Clean-up](#3.-Clean-up)
   1. [Delete the endpoint](#A.-Delete-the-endpoint)
   1. [Delete the model](#B.-Delete-the-model)


## Usage instructions
You can run this notebook one cell at a time (By using Shift+Enter for running a cell).

## Imports

In [4]:
import json
from sagemaker import ModelPackage
from sagemaker import get_execution_role
import sagemaker as sage
import boto3

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


### Check the version of boto3 - must be v1.25.4 or higher
If you see a lower version number, pick another kernel to run the notebook, with Python 3.8 or above

In [ ]:
boto3.__version__

### Install ai21 python SDK

In [5]:
! pip install -U "ai21[AWS]>=2.0.3"
import ai21


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


### Check the version of ai21 - must be 2.0.3 or higher

In [ ]:
import ai21
ai21.__version__

## 1. Select model package
Confirm that you received this notebook from the model catalog in SageMaker AWS Management Console.

In [7]:
region = boto3.Session().region_name

# Get the updated ARN
model_package_arn = ai21.SageMaker.get_model_package_arn(model_name="paraphrase", region=region)

In [8]:
role = get_execution_role()
sagemaker_session = sage.Session()

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


## 2. Create an endpoint and perform real-time inference

If you want to understand how real-time inference with Amazon SageMaker works, see [Documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/deploy-model.html).

In [13]:
endpoint_name = "paraphrase"

content_type = "application/json"

real_time_inference_instance_type = (
    "ml.g4dn.2xlarge"
)

### A. Create an endpoint

In [14]:
 create a deployable model from the model package.
model = ModelPackage(
    role=role, model_package_arn=model_package_arn, sagemaker_session=sagemaker_session
)

 Deploy the model
predictor = model.deploy(
    initial_instance_count=1,
    instance_type=real_time_inference_instance_type,
    endpoint_name=endpoint_name, 
    model_data_download_timeout=3600,
    container_startup_health_check_timeout=600,
)

Once endpoint has been created, you would be able to perform real-time inference.

### B. Paraphrase a single sentence

**AI21 Paraphrase model** offers access to our world-class paraphrasing engine. It has been specifically developed for suggesting alternative ways to convey the same message using different words.

This model takes a piece of text and returns a list of paraphrases that convey the same meaning using different words. It contains features to enable you more control over the output by adjusting the tone and style. Moreover, you can add additional context to the text, to make the rewrite suggestion more relevant (by paraphrasing a certain range of your text).

The input text should contain **no more than 500 characters**. The output will include up to 10 suggestions to paraphrase your input text.

First, create an AI21 Sagemaker client

In [11]:
from ai21 import AI21SageMakerClient
from ai21.models import ParaphraseStyleType

client = AI21SageMakerClient(endpoint_name=endpoint_name)

The following is a simple example of a paraphrase of a single sentence.

In [12]:
#text = "Throughout this page, we will explore the advantages and features of the Paraphrase model."

#response = client.paraphrase.create(
    text=text,
)

print(response.suggestions)

AttributeError: 'NoneType' object has no attribute 'group'

Let's print the suggestions in a more comfortable way:

In [ ]:
print("Original sentence:")
print(text)
print("============================")
print("Suggestions:")
print("\n".join(["- " + x.text for x in response.suggestions]))

### C. Paraphrase with a ceratin style

You have the choice between five different styles:
- General - there are fresh and creative ways to rephrase sentences. Offer them to your users.
- Casual - convey a friendlier and more accessible tone for the right audience.
- Formal - present your words in a more professional way.
- Shorten - express your messages clearly and concisely.
- Expand - expand your sentences to give more detail, nuance and depth.

Below is an example of the same text paraphrased in differnt styles.

In [ ]:
text = "This might be the best thing that has happened to my writing"

response_formal = client.paraphrase.create(
    text=text,
    style=ParaphraseStyleType.FORMAL,
)


response_casual = client.paraphrase.create(
    text=text,
    style=ParaphraseStyleType.CASUAL,
)

In [ ]:
print("Original sentence:")
print(text)
print("=======================")
print("Formal paraphrase:")
print(response_formal.suggestions[0].text)
print("=======================")
print("Casual paraphrase:")
print(response_casual.suggestions[0].text)

### D. Add context to enhance the paraphrasing

You can paraphrase only part of the text while keeping the surrounding text unchanged by specifying a range within the text. This is done by passing to the model the range to paraphrase (by specifying _startIndex_ and _endIndex_).

**AVOID** ranges that start or end mid-word. Cases like this usually yield few or no suggestions.

See the following example for a call that combines a short style and the context feature, i.e. - paraphrase only part of the text, while maintaining the full context. The _startIndex_ can be either the character you wish to paraphrase from, or a whitespace before it.

In [ ]:
full_context = "This classic cotton t-shirt is made from soft, comfortable cotton and is suitable for everyday wear. In addition to providing ultimate comfort and a timeless appearance, this t-shirt has a durable ribbed neckline and a regular fit. It is available in a variety of colors so that you can wear it up or down for any occasion. It's great for running errands or dressing up for a night out."
text_to_paraphrase = "This classic cotton t-shirt is made from soft, comfortable cotton and is suitable for everyday wear."

# Find the range to pass to the model
start_index = full_context.find(text_to_paraphrase)
end_index = start_index + len(text_to_paraphrase)

response = client.paraphrase.create(
    text=full_context,
    style=ParaphraseStyleType.SHORT,
    start_index=start_index,
    end_index=end_index,
)

Note that the response will always include the full text and not just the paraphrased part:

In [ ]:
print("Original sentence:")
print(full_context)
print("=======================")
print("Text to paraphrase:")
print(text_to_paraphrase)
print("=======================")
print("Suggestions:")
print("\n".join(["- " + x.text for x in response.suggestions]))

### Interested in learning more?
Take a look at our [guide](https://docs.ai21.com/docs/paraphrase-api) to understand all the capabilities of AI21 Paraphrase model

## 3. Clean-up

### A. Delete the endpoint

Now that you have successfully performed a real-time inference, you do not need the endpoint any more. You can terminate the endpoint to avoid being charged.

In [ ]:
model.sagemaker_session.delete_endpoint(endpoint_name)
model.sagemaker_session.delete_endpoint_config(endpoint_name)

### B. Delete the model

In [ ]:
model.delete_model()